# Feature Engineering and Model selection

## Setup

In [81]:
import pandas as pd

In [82]:
df = pd.read_csv("../../data/housing_data.csv", index_col="id")

# Missing neighborhood set to NaN
df.neighborhood = df.neighborhood.apply(lambda x: np.nan if x == "BA" else x)

# Drop null prices and areas over 2000m^2
df = df[~(df.prices.isna())]
df = df[~(df.areas > 2000)]

### Our data

In [83]:
df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,neighborhood
id,,,,,,,
2669329904,324.0,5.0,3.0,4.0,340000.0,house,Sao Tome
2667338201,51.0,2.0,1.5,1.0,295000.0,NaN,Santa Teresa
2643720430,94.0,3.0,4.0,2.0,629000.0,apartment,Cidade Jardim
2681509397,66.0,2.0,2.0,2.0,460000.0,apartment,Imbuí
2671493238,82.0,3.0,3.0,2.0,649990.0,apartment,Piatã


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1022 entries, 2669329904 to 2665920996
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   areas         1022 non-null   float64
 1   bedrooms      1021 non-null   float64
 2   bathrooms     1021 non-null   float64
 3   parkingSpots  853 non-null    float64
 4   prices        1022 non-null   float64
 5   type          1017 non-null   object 
 6   neighborhood  999 non-null    object 
dtypes: float64(5), object(2)
memory usage: 63.9+ KB


## Feature Engineering

### Neighborhood Price/Area

Since this relationship behaves differently for apartments/houses. We'll have different values for each. 

In [85]:
neighborhood_house_price_sum =  df[df.type == "house"].groupby("neighborhood").prices.sum()
neighborhood_house_area_sum =  df[df.type == "house"].groupby("neighborhood").areas.sum()

neighborhood_apartment_price_sum =  df[df.type == "apartment"].groupby("neighborhood").prices.sum()
neighborhood_apartment_area_sum =  df[df.type == "apartment"].groupby("neighborhood").areas.sum()

def calculate_neighborhood_area_price(x):
    if x.type == "house" and x.neighborhood in neighborhood_house_price_sum:
        return neighborhood_house_price_sum[x.neighborhood] / neighborhood_house_area_sum[x.neighborhood]
    if x.type == "apartment" and x.neighborhood in neighborhood_house_price_sum:
        return neighborhood_apartment_price_sum[x.neighborhood] / neighborhood_apartment_area_sum[x.neighborhood]
    else: return np.nan

df["neighborhood_area_price"] = df.apply(calculate_neighborhood_area_price, axis=1)

In [86]:
df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,neighborhood,neighborhood_area_price
id,,,,,,,,
2669329904,324.0,5.0,3.0,4.0,340000.0,house,Sao Tome,747.782003
2667338201,51.0,2.0,1.5,1.0,295000.0,NaN,Santa Teresa,NaN
2643720430,94.0,3.0,4.0,2.0,629000.0,apartment,Cidade Jardim,NaN
2681509397,66.0,2.0,2.0,2.0,460000.0,apartment,Imbuí,5906.225980
2671493238,82.0,3.0,3.0,2.0,649990.0,apartment,Piatã,7359.345794


## Model Evaluation

### Prepare training data

In [87]:
df.type = df.type.replace({"house": 0, "apartment": 1})

X = df.drop("prices", axis=1)
y = df.prices

In [88]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV

### Set Pipeline function

In [89]:
def create_pipeline(model, training_data):
    numerical_cols = training_data.select_dtypes(include='number').columns
    categorical_cols = training_data.select_dtypes(include='object').columns

    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='median')

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    #Bundling
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    return Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

### Random Forest Regressor

In [90]:
from sklearn.ensemble import RandomForestRegressor

pipe = create_pipeline(RandomForestRegressor(random_state=1, n_jobs=6), X)

#### Grid Search

In [91]:
params = {
    "model__n_estimators" : [100, 250, 300, 500],
    'model__bootstrap': [True, False]
}

cv = GridSearchCV(pipe, params, cv=5, scoring=["r2", "neg_mean_absolute_error"], refit="r2", verbose=2)

In [92]:
cv.fit(X, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END .....model__bootstrap=True, model__n_estimators=100; total time=   0.4s


[CV] END .....model__bootstrap=True, model__n_estimators=100; total time=   0.4s
[CV] END .....model__bootstrap=True, model__n_estimators=100; total time=   0.3s
[CV] END .....model__bootstrap=True, model__n_estimators=100; total time=   0.3s
[CV] END .....model__bootstrap=True, model__n_estimators=100; total time=   0.3s
[CV] END .....model__bootstrap=True, model__n_estimators=250; total time=   0.9s
[CV] END .....model__bootstrap=True, model__n_estimators=250; total time=   0.9s
[CV] END .....model__bootstrap=True, model__n_estimators=250; total time=   1.0s
[CV] END .....model__bootstrap=True, model__n_estimators=250; total time=   0.9s
[CV] END .....model__bootstrap=True, model__n_estimators=250; total time=   0.8s
[CV] END .....model__bootstrap=True, model__n_estimators=300; total time=   1.1s
[CV] END .....model__bootstrap=True, model__n_estimators=300; total time=   1.2s
[CV] END .....model__bootstrap=True, model__n_estimators=300; total time=   1.5s
[CV] END .....model__bootstr

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='median'),
                                                                         Index(['areas', 'bedrooms', 'bathrooms', 'parkingSpots', 'type',
       'neighborhood_area_price'],
      dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['neighborhood'], dtype='object'))])),
                                       ('model',
                                        RandomForestRegressor(n_jobs=6,
                                                              random_state=1))]),
             param_grid={'model__bootstrap': [True, False],
                         'model__n_estimators': [100, 250, 300, 500]},
             refit='r2', scoring=['r2', 'neg_mean_absolute_error'], verbose=2)

In [93]:
results = pd.DataFrame(cv.cv_results_)
results[["mean_fit_time", "mean_score_time", "param_model__bootstrap", "param_model__n_estimators",
          "mean_test_r2", "rank_test_r2", "mean_test_neg_mean_absolute_error", "std_test_neg_mean_absolute_error"]].sort_values("rank_test_r2")

,mean_fit_time,mean_score_time,param_model__bootstrap,param_model__n_estimators,mean_test_r2,rank_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error
0,0.412215,0.040615,True,100,0.793749,1,-48173.139775,6391.149572
2,1.342743,0.072026,True,300,0.792499,2,-47972.302853,5938.228594
3,1.956243,0.123067,True,500,0.792011,3,-48069.722494,5875.808953
1,0.878223,0.098688,True,250,0.791862,4,-47994.660539,5880.884523
6,1.496066,0.070053,False,300,0.762920,5,-53406.973698,7918.189482
4,0.539491,0.035692,False,100,0.762702,6,-53479.276829,7940.120373
5,1.237598,0.062605,False,250,0.762390,7,-53426.029668,7918.806674
7,2.386719,0.111589,False,500,0.762178,8,-53446.968739,7873.215065


Best: bootstrap - True; estimators - 100; r2 - 0.793749; neg_mean_absolute_error - -48173.139775; std_test_neg_mean_absolute_error - 6391.149572

### XGBoost

In [94]:
from xgboost import XGBRegressor

pipe = create_pipeline(XGBRegressor(random_state=1, n_jobs=4), X)

#### GridSearchCV

In [95]:
params = {
    "model__n_estimators" : [200, 300, 500],
    'model__learning_rate': [0.01, 0.05, 0.1],
    "model__max_depth" : [5, 6, 7, None]
}

cv = GridSearchCV(pipe, params, cv=5, scoring=["r2", "neg_mean_absolute_error"], refit="r2", verbose=2)

In [96]:
cv.fit(X, y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=200; total time=   0.1s


[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=200; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=200; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=200; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=200; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=300; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=300; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=300; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=300; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=300; total time=   0.0s
[CV] END model__learning_rate=0.01, model__max_depth=5, model__n_estimators=500; total time=   0.2s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='median'),
                                                                         Index(['areas', 'bedrooms', 'bathrooms', 'parkingSpots', 'type',
       'neighborhood_area_price'],
      dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown=...
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=4,
                                                     num_parallel_tree=None,
                                                     random_state=1, ...))]),
             param_grid={'model__learning_rate': [0.01, 0.05, 0.1],
                         'model__max_depth': [5, 6, 7, None],
                         'model__n_estimators': [200, 300, 500]},
             refit='r2', scoring=['r2', 'neg_mean_absolute_error'], verbose=2)

In [97]:
results = pd.DataFrame(cv.cv_results_)
results[["mean_fit_time", "mean_score_time", "param_model__n_estimators", "param_model__learning_rate", "param_model__max_depth",
          "mean_test_r2", "rank_test_r2","mean_test_neg_mean_absolute_error", "std_test_neg_mean_absolute_error"]].sort_values("rank_test_r2").head()

,mean_fit_time,mean_score_time,param_model__n_estimators,param_model__learning_rate,param_model__max_depth,mean_test_r2,rank_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error
24,0.092562,0.007705,200,0.1,5,0.798123,1,-47258.508938,8326.933060
14,0.164322,0.008141,500,0.05,5,0.798060,2,-47376.391054,8533.855535
25,0.124016,0.007667,300,0.1,5,0.797736,3,-47539.986904,8616.497098
13,0.107333,0.007411,300,0.05,5,0.795974,4,-47492.204659,7971.479286
26,0.169811,0.008009,500,0.1,5,0.795664,5,-48055.879109,8789.645890


Best: estimators - 200; learning_rate - 0.1; max_depth - 5 ; r2 - 0.798123; neg_mean_absolute_error - -47258.508938; std_test_neg_mean_absolute_error - 8326.933060